In [2]:
#find all FVG among given candles, and retur datetime stamps of each event

# iterate through window of 3 items, flag those where 


candle1 = (100, 110, 95, 105)  # First candle
candle2 = (106, 115, 105, 110)  # Second candle (ignored for gap calculation)
candle3 = (115, 120, 112, 119)  # Third candle

# Simplified Gap Calculation
# Calculating both potential gaps and selecting the maximum as the effective gap

gap_bullish = abs(candle1[1] - candle3[2])  # High of the first - Low of the third
gap_bearish = abs(candle1[2] - candle3[1])  # Low of the first - High of the third

# Select the larger gap
effective_gap = min(gap_bullish, gap_bearish)

effective_gap

2